In [1]:
using Rocket
using MacroTools
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.
┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1278
┌ Warning: Package GraphPPL does not have ReactiveMP in its dependencies:
│ - If you have GraphPPL checked out for development and have
│   added ReactiveMP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with GraphPPL
└ Loading ReactiveMP into GraphPPL from project dependency, future 

In [65]:
@capture(:(s = r(;)), lhs_ = rhs_)

true

In [67]:
dump(rhs)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol r
    2: Expr
      head: Symbol parameters
      args: Array{Any}((0,))


In [6]:
@model function coin_model()
    
    for i in 1:10 
        θ ~ Beta(1, 1) + 1
    end
    
    return 1
end

LoadError: syntax: invalid syntax ; 

In [12]:
@model function coin_model()
    
    for i in 1:10 
        θ ~ (Beta(1, 1) where { q = MeanField() }) + 1
    end
    
    return 1
end

LoadError: syntax: invalid syntax ; q = MeanField()

In [14]:
MacroTools.prettify(@macroexpand @model function coin_model()
    
    for i in 1:10 
        θ ~ (Beta(1, 1) where { q = MeanField() }) + 1
    end
    
    return 1
end)

:(function coin_model(; options = NamedTuple{()}(()))
      snake = Model(options)
      for i = 1:10
          (pigeon, θ) = ReactiveMP.make_node(snake, +, ReactiveMP.AutoVar(:θ), ReactiveMP.as_variable(snake, Symbol("##arg#325"), begin
                          (okapi, goldfinch) = ReactiveMP.make_node(snake, Beta, ReactiveMP.AutoVar(Symbol("##nvar#321")), ReactiveMP.as_variable(snake, Symbol("##arg#323"), 1), ReactiveMP.as_variable(snake, Symbol("##arg#324"), 1); $(Expr(:(=), :factorisation, :(MeanField()))))
                          goldfinch
                      end), ReactiveMP.as_variable(snake, Symbol("##arg#326"), 1); )
      end
      activate!(snake)
      return (snake, 1)
      error("'return' statement is missing")
  end)

In [47]:
function coin_model(; options = NamedTuple{()}(()))
      bee = Model(options)
      (camel, θ) = ReactiveMP.make_node(bee, Beta, ReactiveMP.AutoVar(:θ), ReactiveMP.as_variable(bee, Symbol("##arg#367"), $(Expr(:parameters))), ReactiveMP.as_variable(bee, Symbol("##arg#368"), 1), ReactiveMP.as_variable(bee, Symbol("##arg#369"), 1); )
      activate!(bee)
      return (bee, 1)
      error("'return' statement is missing")
  end

LoadError: syntax: "$" expression outside quote around In[47]:3

In [37]:
model = Model()
a = datavar(:a, Dirac{Float64})
node, var = ReactiveMP.make_node(model, sqrt, ReactiveMP.AutoVar(Symbol("##nvar#264")), ReactiveMP.as_variable(model, Symbol("##arg#267"), a); )

(nothing, DataVariable{Any,ProxyObservable{Message,Rocket.CombineLatestObservable{Tuple{Any},Tuple{ProxyObservable{Any,Subject{Message{Dirac{Float64}},AsapScheduler,AsapScheduler},Rocket.MapProxy{Message{Dirac{Float64}},typeof(getdata)}}},PushNew},Rocket.MapProxy{Tuple{Any},ReactiveMP.var"#401#403"{typeof(sqrt)}}}}(Symbol("##nvar#264"), ProxyObservable(Message, MapProxy(Tuple{Any})), ReactiveMP.DataVariableProps(0)))

In [38]:
model, (y, a, b, θ) = coin_model(options = (default_factorisation = FullFactorisation(), ));

LoadError: Unknown functional form 'sqrt' used for node specification.

In [17]:
getdata(model)

Dict{Symbol,DataVariable} with 4 entries:
  :a                   => DataVariable{Dirac{Float64},Subject{Message{Dirac{Flo…
  :b                   => DataVariable{Dirac{Float64},Subject{Message{Dirac{Flo…
  :y                   => DataVariable{Dirac{Float64},Subject{Message{Dirac{Flo…
  Symbol("##nvar#275") => DataVariable{Any,ProxyObservable{Message,CombineLates…

In [26]:
Base.return_types(sqrt, (Float64, ))

1-element Array{Any,1}:
 Float64

In [11]:
model_options(NamedTuple{()}(()))

ModelOptions{DefaultMessageGate,DiscontinueStreamTransformer,FullFactorisation}(DefaultMessageGate(), DiscontinueStreamTransformer(), FullFactorisation())

In [26]:
using MacroTools

In [29]:
@capture(:(a[1]), a_Symbol)

false

In [1]:
import MacroTools: @capture

In [7]:
@capture(:(a), s_Symbol)

LoadError: UndefVarError: MacroTools not defined